Obviously we want a description of the convec hull of the vectors in $B$. Therefore we want a description of the vertecies of the convex hull in the shape:

$$\vec{n_i}^T x + a_i \leq 0 $$

We can combine these using the Matrix vector notation:

$$N^T x + a\leq 0 $$

To get this, we can use the scipy package.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from scipy.spatial import ConvexHull, convex_hull_plot_2d

We want to da a first test of the capabilities:

In [ ]:
points = np.random.rand(5, 2)   # 30 random points in 2-D
hull = ConvexHull(points)
plt.plot(points[:,0], points[:,1], 'o')
for simplex in hull.simplices:
    plt.plot(points[simplex, 0], points[simplex, 1], 'k-')

Using 'hull.equations' we can get an matrix shape with $(nfacet, ndim+1)$ formated as $[N^T,a]$.


In [ ]:
# get N and a
E = hull.equations
N=E[:,:-1]
a=E[:,-1]
a.shape=[len(a),1]

#setup an heatmap to ilustrate it
x=np.linspace(0,1,100)
y=np.linspace(0,1,100)
X,Y=np.meshgrid(x,y)

X.shape=(1,100**2)
Y.shape=[1,100**2]

O = (N@np.concatenate((X,Y),axis = 0))+a<0
W = np.prod(O,axis=0)
#W = O[3,:]

X.shape=[100,100]
Y.shape=[100,100]
W.shape=[100,100]


cmap = plt.get_cmap('PiYG')
cf = plt.contourf(X,Y,W,cmap=cmap,vmin=0, vmax=1)
plt.colorbar(cf)
for simplex in hull.simplices:
    plt.plot(points[simplex, 0], points[simplex, 1], 'k-')


In [ ]:
hull.equations

But unfotunatley Qhull does not work if alll the points are on a hyperplane, or if there are not enough points provided.

In [ ]:
#Try the hull of two points 
punkte = np.array([[1,0,0],[-1,0,0],[1,1,0],[0,0,0]])
hull = ConvexHull(punkte)

In [ ]:
plt.plot(punkte[:,0], punkte[:,1],'o')

Because it is quite common that we have multiple $b$ on a hyperplane (e.g. if they have the same order) we have to find a way to get it work.

For this i propose the following. 

1) We transform the $b_i$ such that: $b_1 = 0$. This makes further computation easier
 
 $$B_{shifted} = B - b_1$$
 
2) We test the dimentionality of the span of $B_{shifted}$ using the SVD. 

If the SVD is $B_{shifted} = USV.T$ with $S = diag(\sigma_1,\sigma_2,\cdots,\sigma_r,0\cdots,0)$

The vectors $u_1,\cdots,u_r$ are the span of $B$. This ist the subspace in whitch the posddible values of $b$ are.
To make use of chull we want to transform this subspace on a the $R^{r}$. For this we compute $B_{transformed} = [u_1,\cdots,u_r].T B_{shifted}$

By calling 'Chull' we get the normal vectors in the transformed space $N_{transformed}$. To transform these back we undo the transformation by computing $N_{hull} = [u_1,\cdots,u_r] N_{transformed}$. This transformation laves the $a$ unchanged.

Now we have to constrict the possible values in the shifted space to $Span(u_1,\cdots,u_r)$. This can be done by adding additional normal vectors. By using the none reduced form of the SVD we get the according normal vectors with $N_{subspace} = [u_{r+1},\cdots,u_s]$ ($U \in R^{s \times s}$). Additionally we get the vector $a_{subspace shifted} = [0 \times (s-r)].T$ These are equality constraints, so we have to treat them seperatly (We could also add inequalities with $-N_{subspace}$ but this is probably not adventagious)



In [ ]:
B = np.array([[1,2,1],[3,4,1],[5,7,1]]).T
#b_p = np.array([1,4,4])
#B = np.array([b_p , 2*b_p , 3*b_p]).T
display(B)
b_0 = B[:,0]
b_0.shape=(len(b_0),1)
B_shifted = B-b_0
display(B_shifted)

In [ ]:
U,S,V = np.linalg.svd(B_shifted)
N_dim = np.linalg.matrix_rank(B_shifted)
print(N_dim)
display(U)
display(S)
display(V)

In [ ]:
print(np.linalg.norm(U[:,1]))
print(U[:,0].T@U[:,2])

In [ ]:
b_transformed = U[:,0:N_dim].T@B_shifted
display(b_transformed)

In [ ]:
hull = ConvexHull(b_transformed.T)
E = hull.equations
N_transformed=E[:,:-1].T
a_hull_sh=E[:,-1]
a_hull_sh.shape=[len(a_hull_sh),1]
display(N)
display(a_hull_sh)

In [ ]:
N_hull=U[:,0:N_dim]@N_transformed
display(N_hull)

In [ ]:
N_subspace =U[:,N_dim:]
display(N_subspace)
a_subspace_sh = np.zeros((len(b_0)-N_dim,1))
display(a_subspace_sh)
#N_trback.T@B_shifted+a


Now we have to unshift the edges. For his we have to consider

$$\vec{n_i}^T x + a_i \leq 0 $$

We have split up the original coordiante such as $x = b_i -b_1$

$$ \vec{n}^T (b_i -b_1)^T + a_i \leq 0 $$

$$ \vec{n}^T b_i - \vec{n}^T b_1 + a_i \leq 0 $$
$$ \vec{n}^T b_i + (- \vec{n}^T b_1 + a_i) \leq 0 $$


In [ ]:
a = a_hull_sh-N_hull.T@b_0
display(a)
a_subspace = a_subspace_sh-N_subspace.T@b_0
display(a_subspace)

In [ ]:
#Now test it on the points
display(np.round(N_hull.T@B+a,8))
display(np.round(N_subspace.T@B+a_subspace,8))

Additional remarks: 
If the remaining dimention is only 1D (all points are one line) Qhull does also not work, but it is trivial to get n and a for this case 


In [ ]:
def Eqsys_line(B):
    N = np.array([[-1.,1.]])
    a = np.array([[np.min(B)],[-np.max(B)]])
    if a[0]+a[1] == 0:
        print('only single Point, not able to compute convex hull')
        raise ValueError
    return N,a

#Test
b = np.array([[1,4]])
N,a = Eqsys_line(b)

b_test = np.array([np.arange(-5,6)])

display(np.prod((N.T@b_test+a)<=0,axis=0))
display(np.array([b_test[0,:],np.prod((N.T@b_test+a)<=0,axis=0)]))

In [ ]:
# Here the implementation inside a general function
def Eqsys_chull(B):
    """
    Function to compute the equarionsystem defining a c_hull of the points b_1,...,b_l
    
    Parameters: 
    B = [b_1,...,b_l] Points that span the chull
    
    Returns:
    Two sets of constriants 
        N_hull^T x     + a_hull    <= 0 
        N_subspace^T x + a_subspace = 0
    
    N_hull = [n_1,...,n_k] Matrix with normal vectors of the inequality constraints
    a_hull = [a_r,...,a_k] Vector containig the offsets of the inequality constraints
    N_subspace = [n_k+1,...,n_k+(r-s)] Matrix with normal vectors of the equality constraints
    a_subspace = [n_k+1,...,n_k+(r-s)] Vector containig the offsets of the equality constraints
    """
    b_0 = B[:,0]
    s = len(b_0)
    b_0.shape=(s,1)
    B_shifted = B-b_0
    
    N_dim = np.linalg.matrix_rank(B_shifted)
    if N_dim == s: #No need to transform using SVD, we can simply use qhull
        if N_dim == 1:
            N_hull,a_hull_sh = Eqsys_line(B_shifted)
        else:
            hull = ConvexHull(B_shifted.T)
            E = hull.equations
            N_hull=E[:,:-1].T
            a_hull_sh=E[:,-1]
            a_hull_sh.shape=[len(a_sh),1]
        
        N_subspace = np.array([[]])
        a_subspace_sh = np.array([[]])
        
        
    if N_dim <= s: #Transform using SVD
        print('Transform, n_dim =',N_dim)
        U,S,V = np.linalg.svd(B_shifted)
        B_transformed = U[:,0:N_dim].T@B_shifted
        if N_dim == 1:
            N_transformed,a_hull_sh = Eqsys_line(B_transformed)
        else:
            hull = ConvexHull(B_transformed.T)
            E = hull.equations
            N_transformed=E[:,:-1].T
            a_hull_sh=E[:,-1]
            a_hull_sh.shape=[len(a_hull_sh),1]
        
        
        N_hull=U[:,0:N_dim]@N_transformed

        N_subspace =U[:,N_dim:]
        a_subspace_sh = np.zeros((s-N_dim,1))

    a_hull = a_hull_sh-N_hull.T@b_0
    a_subspace = a_subspace_sh-N_subspace.T@b_0
    
    return (N_hull,a_hull,N_subspace,a_subspace)


In [ ]:
"""First Test:
Goals: 
General test of errors in Implemntation (signs, interplay between functions)
Test hull of two points

using 2D-space with 3 point in one line
"""

#define points: 
b_1 = np.array([1,3])
b_2 = np.array([2,4])
b_3 = np.array([-1,1])

B = np.array([b_1,b_2,b_3]).T
(N_hull,a_hull,N_subspace,a_subspace) = Eqsys_chull(B)

#first tests for reasonability:
print('shape of N_hull should be [2,2] it is:',N_hull.shape)
print('shape of a_hull should be [2,1] it is:',a_hull.shape)
print('shape of N_subspace should be [2,1] it is:',N_subspace.shape)
print('shape of a_subspace should be [1,1] it is:',a_subspace.shape)

print('Test subspace: N_subspace^T B+a_subspace should be [0,0,0] it is:',np.round(N_subspace.T@B+a_subspace,5))

#setup an heatmap to ilustrate it
x=np.linspace(-2,3,100)
y=np.linspace(0,5,100)
X,Y=np.meshgrid(x,y)

X.shape=(1,100**2)
Y.shape=[1,100**2]

v = np.concatenate((X,Y),axis = 0)
W = np.all( np.concatenate((N_hull.T@v+a_hull<0,np.abs(N_subspace.T@v+a_subspace)<0.001),axis=0),axis=0)
#W = O[3,:]

X.shape=[100,100]
Y.shape=[100,100]
W.shape=[100,100]

%matplotlib inline
cmap = plt.get_cmap('PiYG')
cf = plt.contourf(X,Y,W,cmap=cmap,vmin=0, vmax=1)

#plot points 
plt.plot(B[0,:],B[1,:],'o')

In [ ]:
"""Second Test:
Goals: 
General test of errors in Implemntation (signs, interplay between functions)
Test transformation of results of Qhull

using 3D-space with 3 points in one plane
"""

#We start of using 4 point on the x-y-plane
b_0_ = np.array([-1,3,0])
b_1_ = np.array([1,0,0])
b_2_ = np.array([-2,1,0])
b_3_ = np.array([-2,2,0])


B = np.array([b_0_,b_1_,b_2_,b_3_]).T

#Now we shift these onto the plane z = x-2y+1
B[2,:]= B[0,:]-2*B[1,:]+1
print(B)

(N_hull,a_hull,N_subspace,a_subspace) = Eqsys_chull(B)

#first tests for reasonability:
print('shape of N_hull should be [3,4] it is:',N_hull.shape)
print('shape of a_hull should be [4,1] it is:',a_hull.shape)
print('shape of N_subspace should be [3,1] it is:',N_subspace.shape)
print('shape of a_subspace should be [1,1] it is:',a_subspace.shape)

print('Test subspace: N_subspace^T B+a_subspace should be [0,0,0] it is:',np.round(N_subspace.T@B+a_subspace,5))
print('Test points in hull:',np.all(N_hull.T@B+a_hull<1e-10))

#We now add a 3D plot, we choose X and Y on a grid and choose z such that N_subspace.T @ x +a_subspace = 0

x=np.linspace(-3,2,100)
y=np.linspace(-1,4,100)
X,Y=np.meshgrid(x,y)

X.shape=(1,100**2)
Y.shape=[1,100**2]

#N_subspace[0,:]*X+N_subspace[1,:]*Y+N_subspace[2,:]*Z + a_subspace[0,:] = 0
Z = -(N_subspace[0,:]*X+N_subspace[1,:]*Y + a_subspace)/N_subspace[2,:]

v = np.concatenate((X,Y,Z),axis = 0)
#Test if subspace is correct 
print('Test if subspace is correct:', np.all(np.abs(N_subspace.T@v+a_subspace)<0.001))

W = np.all(np.concatenate((N_hull.T@v+a_hull<0,np.abs(N_subspace.T@v+a_subspace)<0.001),axis=0),axis=0)


X.shape=[100,100]
Y.shape=[100,100]
Z.shape=[100,100]
W.shape=[100,100]




%matplotlib qt
fig = plt.figure()
ax = fig.gca(projection='3d')

color = np.where(W,'b','grey')
surf = ax.plot_surface(X, Y, Z, cmap=cmap,facecolors=color,
                       linewidth=0, antialiased=False)

ax.plot(B[0,:],B[1,:],B[2,:],'og',zorder=3)



In [ ]:
"""Thirs Test:
Goals: 
Test multiple points and their properites
"""

#1D in 6D 
vec = np.random.rand(4, 1)
line = np.random.rand(6,1)
ap = np.random.rand(6,1)
B = (line@vec.T)+ap

#create points in line but outside of hull
O = line@np.array([[np.min(vec)-0.1],[np.max(vec)+0.1]]).T+ap
#create points that would be inside of hull but move them out of line
perturb = (np.eye(6)-(line@line.T)/(line.T@line))@np.random.rand(6,4)
if np.any(np.linalg.norm(perturb,axis=0)< 1e-8):
    print('Testvetokr muss neu erzeugt werden')
V = B+perturb

(N_hull,a_hull,N_subspace,a_subspace) = Eqsys_chull(B)

#first tests for reasonability:
print('shape of N_hull should be [6,2] it is:',N_hull.shape)
print('shape of a_hull should be [2,1] it is:',a_hull.shape)
print('shape of N_subspace should be [6,5] it is:',N_subspace.shape)
print('shape of a_subspace should be [5,1] it is:',a_subspace.shape)

print('Test subspace: N_subspace^T B+a_subspace should be [0,..,0] it is:',np.all((N_subspace.T@B+a_subspace))<1e-12)
print('Test points in hull:',np.all(N_hull.T@B+a_hull<1e-12))

print('Test point outside of hull:',not np.any(np.all(N_hull.T@O+a_hull<1e-12,axis=0)))
if np.any(np.linalg.norm(perturb,axis=0)< 1e-8):
    print('Testvectro to small, please rerun')
print('Test point outside of subspace', not np.any(np.all(np.abs(N_subspace.T@V+a_subspace)<1e-12,axis =0)))


#2D in 6D 
points = np.random.rand(5, 2)
plane = np.random.rand(6,2)
ap = np.random.rand(6,1)
B = plane@points.T+ap

(N_hull,a_hull,N_subspace,a_subspace) = Eqsys_chull(B)

#first tests for reasonability:
print('shape of N_hull should be [6,?] it is:',N_hull.shape)
print('shape of a_hull should be [?,1] it is:',a_hull.shape)
print('shape of N_subspace should be [6,4] it is:',N_subspace.shape)
print('shape of a_subspace should be [4,1] it is:',a_subspace.shape)

print('Test subspace: N_subspace^T B+a_subspace should be [0,..,0] it is:',np.all((N_subspace.T@B+a_subspace))<1e-10)
print('Test points in hull:',np.all(N_hull.T@B+a_hull<1e-10))




#5D in 6D 
points = np.random.rand(10, 5)
hyperplane = np.random.rand(6,5)
ap = np.random.rand(6,1)
B = hyperplane@points.T+ap

(N_hull,a_hull,N_subspace,a_subspace) = Eqsys_chull(B)

#first tests for reasonability:
print('shape of N_hull should be [6,?] it is:',N_hull.shape)
print('shape of a_hull should be [?,1] it is:',a_hull.shape)
print('shape of N_subspace should be [6,1] it is:',N_subspace.shape)
print('shape of a_subspace should be [1,1] it is:',a_subspace.shape)

print('Test subspace: N_subspace^T B+a_subspace should be [0,..,0] it is:',np.all((N_subspace.T@B+a_subspace))<1e-10)
print('Test points in hull:',np.all(N_hull.T@B+a_hull<1e-10))


In [ ]:
np.linalg.norm(perturb,axis=0)

In [ ]:
line.T@perturb

In [ ]:
(N_subspace.T@V+a_subspace)

In [ ]:
(N_subspace.T@V+a_subspace)<1e-12